In [ ]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.2/172.2 KB 10.9 MB/s eta 0:00:00


In [ ]:
from keras.datasets import cifar10

((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0
#trainX = trainX.reshape((trainX.shape[0], 3072))
#testX = testX.reshape((testX.shape[0], 3072))


170498071/170498071 [==============================] - 13s 0us/step


In [ ]:
from keras.utils import to_categorical

testY = to_categorical(testY)
trainY = to_categorical(trainY)

In [ ]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers import Dense
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten, Dropout
from keras.optimizers import SGD


def build_model(hp):
  input_shape = trainX[0].shape # (32,32,3) 

  model = Sequential()
  model.add(Conv2D(32, kernel_size=(3,3), activation="relu", input_shape=input_shape))
  model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2) ))
  model.add(Dropout(hp.Float("dp1", min_value=0.1, max_value=0.5, step=0.1)))
  model.add(Flatten())
  model.add(Dense(128, activation="relu"))
  model.add(Dropout(hp.Float("dp2", min_value=0.1, max_value=0.5, step=0.1)))
  model.add(Dense(10, activation="Softmax"))

  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  sgd = SGD(learning_rate)
  model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

  print(model.summary())
  return model

In [ ]:
import keras_tuner
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="helloworld",
)
tuner.search_space_summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)               1605760   
                                                        

In [ ]:
#algo = model.fit(trainX, trainY, validation_data=(testX, testY), epochs=100, batch_size=32) 
tuner.search(trainX, trainY, epochs=2, validation_data=(testX, testY))

Trial 3 Complete [00h 00m 41s]
val_accuracy: 0.3219500035047531

Best val_accuracy So Far: 0.4631499946117401
Total elapsed time: 00h 02m 23s


In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build()
best_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 64)        0         
                                                                 
 flatten (Flatten)           (None, 12544)             0         
                                                                 
 dense (Dense)               (None, 128)               1605760   
                                                        

In [ ]:
tuner.results_summary()

Results summary
Results in my_dir/helloworld
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 1 summary
Hyperparameters:
dp1: 0.1
dp2: 0.30000000000000004
lr: 0.009947201618750954
Score: 0.4631499946117401

Trial 2 summary
Hyperparameters:
dp1: 0.4
dp2: 0.4
lr: 0.0014273272133361508
Score: 0.3219500035047531

Trial 0 summary
Hyperparameters:
dp1: 0.1
dp2: 0.30000000000000004
lr: 0.00045130754049079694
Score: 0.27140000462532043


In [ ]:
best_hps = tuner.get_best_hyperparameters(5)
print(best_hps)

[<keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7f3280dbe3a0>, <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7f3280e56f40>, <keras_tuner.engine.hyperparameters.hyperparameters.HyperParameters object at 0x7f33ec1d3fa0>]


# https://keras.io/guides/keras_tuner/getting_started/

# Tunner con el dataset IRIS

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
dataX, dataY = data['data'], data['target']
dataY = to_categorical(dataY)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split( dataX, dataY, test_size=0.2, shuffle=True)

In [ ]:
from keras.optimizers import SGD, Adam

def build_model(hp):
    model = Sequential()

    # El numero de capas ocultas es definido como hp, entre 1 a 3
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add( 
            Dense(
                # el numero de neuronas de cada capa es definido como hp entre 32 y 128
                units=hp.Int(f"units_{i}", min_value=32, max_value=128, step=32),
                activation="relu"),
            )
        
    if hp.Boolean("dropout"):
        model.add(Dropout(rate=0.25))
    
    model.add(Dense(3, activation="softmax"))
    
    
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        #optimizer=SGD(learning_rate=learning_rate),
        loss="categorical_crossentropy",
        metrics=["accuracy"],
    )
    return model

In [ ]:
import keras_tuner
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=30,
    executions_per_trial=2,
    overwrite=True,
    directory="my_dir",
    project_name="random",
)
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [ ]:
tuner.search(X_train, Y_train, epochs=2, validation_data=(X_test, Y_test))

Trial 30 Complete [00h 00m 02s]
val_accuracy: 0.5499999970197678

Best val_accuracy So Far: 0.8833333253860474
Total elapsed time: 00h 01m 01s


In [ ]:
# Get the top 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
# Build the model.
# Needed for `Sequential` without specified `input_shape`.
best_model.build()
best_model.summary()